<a href="https://colab.research.google.com/github/HidekiAI/ML-manga109-OCR/blob/trunk/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Load minimal pyenv

Skip this if running on CoLab, only useful for local Jupyter-Notebook

In [ ]:
#!/bin/bash
%pip install google-colab

## Libs

-   Ultralitics YOLO

In [ ]:
#!/bin/bash
%pip install ultralytics

For CoLab, you must first make sure remote drive is mounted. To align BASH and Python scripts to work on multiple platform, for local, you'd need to either soft-link (or junction) and/or mount (i.e. `mount bind`).

Note that below is ONLY necessary for Google CoLab to access your Google Drive. If on Notepad/Jupyter, do the following instead (not exact, just the example):

-   Linux: make sure to `ln -sv ~/Google/MyDrive /content/drive` to softlink your Google G-Drive as `/content/drive`
-   Windows: From DOS Command Prompt (right clock to launch as Admin) `mklink.exe /D "C:/content/drive" "C:/Users/HidekiAI/Google/MyDrive/"` to create a dir-junction


In [ ]:
#!/usr/bin/python
# No need to execute this if running locally, this is only for Google CoLab usage
import os
from google.colab import drive

drive.mount('/content/drive')
print(f"Current directory: {os.getcwd()}")

mount_drive = '/content/drive/MyDrive'
if not os.path.exists(mount_drive):
    print(f"Mount drive '{mount_drive}' does not exist")
    
dir_content = os.listdir(mount_drive)
print(f"Directory content: {dir_content}")

## Constants

Where are my data, where do I save my trained data and progress


In [ ]:
#!/usr/bin/python
import os

global mount_drive, root_src_data_path, root_dst_data_path, train_path, val_path, test_path, data_yaml_file_path
mount_drive = '/content/drive/MyDrive'
root_src_data_path = mount_drive + '/projects/ML-manga-ocr-rust/data/text_detection/'
root_dst_data_path = mount_drive + '/projects/ML-manga-ocr-rust/data/text_detection/'
train_path = os.path.join(root_src_data_path, 'images/train')
val_path = os.path.join(root_src_data_path, 'images/val')
test_path = os.path.join(root_src_data_path, 'images/test')
data_yaml_file_path = os.path.join(root_dst_data_path, 'data.yaml')

if not os.path.exists(mount_drive):
    print(f"Mount drive {mount_drive} does not exist")


# NOTE: YOLO will set default value for 'save_dir=runs/detect/train' (if resume=False) and/or 'save_dir=YOLOv8/yolov8s' (if resume=True)
# engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, mixup=0.0, copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, tracker=botsort.yaml, save_dir=runs/detect/train
# print CURRENT directory:
print(f"Current directory: {os.getcwd()}")  # on CoLab, this will print '/content', as in '/content/YOLOv8/yolov8s/weights'

# ls /content/drive
dir_content = os.listdir(mount_drive)
print(f"Directory content: {dir_content}")

# if cwd starts with "C:\", then we are in Windows, so switch paths
if os.getcwd().startswith("C:\\") or os.getcwd().startswith("c:\\"):
    # NOTE: This WILL NOT work if anywhere in the script, there is a call to `os.chdir()`!
    root_src_data_path = '../../data/text_detection'
    root_dst_data_path = './data/'
    train_path = os.path.join(root_src_data_path, 'images/train')
    val_path = os.path.join(root_src_data_path, 'images/val')
    test_path = os.path.join(root_src_data_path, 'images/test')
    data_yaml_file_path = os.path.join(root_dst_data_path, 'data.yaml')

print(f"Current directory: {os.getcwd()}")

# validate paths exist
if not os.path.exists(root_src_data_path):
    print(f"Source data path {root_src_data_path} does not exist")
if not os.path.exists(root_dst_data_path):
    print(f"Destination data path {root_dst_data_path} does not exist")
if not os.path.exists(train_path):
    print(f"Train path {train_path} does not exist")
if not os.path.exists(val_path):
    print(f"Validation path {val_path} does not exist")
if not os.path.exists(test_path):
    print(f"Test path {test_path} does not exist")

## data.yaml

YAML config for YOLO; note that because it's YAML file, it's not based on env-vars or globals


In [ ]:
#!/usr/bin/python

data_yaml_content = f"""
train: {train_path}
val: {val_path}
test: {test_path}

nc: 1  # number of classes
names: ['text']  # class names
"""

with open(data_yaml_file_path, 'w') as f:
    f.write(data_yaml_content)

# verify file now exists:
if not os.path.exists(data_yaml_file_path):
    print(f"Data yaml file {data_yaml_file_path} does not exist")

# Dump yaml content to verify, by reading it back
with open(data_yaml_file_path, 'r') as f:
    print(f.read())

## Training

```bash
yolo detect train data=data.yaml epochs=50 imgsz=640
```

Usage of Model size:

-   `yolov8n.pt` (Nano): The smallest model, optimized for speed and efficiency on resource-constrained devices. It has the least number of parameters and computational complexity, making it fast but less accurate.
-   `yolov8s.pt` (Small): A small model that offers a good balance between speed and accuracy. Suitable for scenarios where both performance and accuracy are important but resource usage needs to be moderate.
-   `yolov8m.pt` (Medium): A medium-sized model that improves accuracy over the small model but at the cost of additional computational resources and slower inference times.
-   `yolov8l.pt` (Large): A larger model with more parameters and higher computational requirements, offering higher accuracy but slower inference times.
-   `yolov8x.pt` (Extra Large): The largest model with the highest number of parameters and computational requirements. It provides the best accuracy but is the slowest in terms of inference speed.


In [ ]:
#!/usr/bin/python

########################### NOTE: ###########################
# This seems to only work on CoLab and not locally (maybe because
# I lack GPU, maybe because I was testing on Windows, etc)
#############################################################

from ultralytics import YOLO
# If `resume=False`:
# engine/trainer: task=detect, mode=train, model=yolov8s.pt, 
#   data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml, epochs=50, 
#   time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, 
#   workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, 
#   seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, 
#   amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, 
#   dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, 
#   half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, 
#   augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, 
#   save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 
#   show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, 
#   dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937, 
#   weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, 
#   dfl=1.5, pose=12.0, kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, 
#   translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, 
#   mixup=0.0, copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, 
#   tracker=botsort.yaml, save_dir=runs/detect/train
# If `resume=True`:
# engine/trainer: task=detect, mode=train, model=yolov8s.pt, 
#   data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml, 
#   epochs=500, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, 
#   cache=disk, device=cpu, workers=8, project=YOLOv8, name=yolov8s, exist_ok=False, 
#   pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, 
#   rect=False, cos_lr=False, close_mosaic=10, resume=yolov8s.pt, amp=True, fraction=1.0, 
#   profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=False, 
#   val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, 
#   half=True, dnn=False, plots=False, source=ultralytics/assets/, vid_stride=1, stream_buffer=False, 
#   visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 
#   show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 
#   show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, 
#   keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, 
#   nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.001, warmup_epochs=3.0, 
#   warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0, 
#   label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, 
#   scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, mixup=0.0, 
#   copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, tracker=botsort.yaml, 
#   image_weights=False, hide_labels=False, hide_conf=False, line_thickness=3, fl_gamma=0.0, v5loader=True, 
#   save_dir=YOLOv8/yolov8s

# NOTE: YOLO will set default value for 'save_dir=runs/detect/train' (if resume=False) and/or 'save_dir=YOLOv8/yolov8s' (if resume=True)
# print CURRENT directory:
print(f"Current directory: {os.getcwd()}")  # on CoLab, this will print '/content', as in '/content/YOLOv8/yolov8s/weights'

# Load the YOLO model
if os.path.exists('last.pt'):
    found_last = True
    model = YOLO('last.pt')
else:
    found_last = False
    # Note that YOLO will default download to `/content/yolov8s.pt` (on CoLab) because current directory is `/content`
    model = YOLO('yolov8s.pt')  # see MD coment above for other versions

# dump data.yaml content
with open(data_yaml_file_path, 'r') as f:
    print(f.read())

# Train the model, assume we don't have any GPU available
model.train(data=data_yaml_file_path, epochs=50, imgsz=640, resume=found_last, device="cpu")
